<a href="https://colab.research.google.com/github/HarshithMulakala/starter-express-api/blob/main/t5_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.2 MB/s eta 0:00:00


In [12]:
import os
import pandas as pd

# Step 1: Generate captions.txt file with token filtering
base_path = "/content/drive/My Drive/LaVi-Bridge"
output_file = "/content/drive/My Drive/LaVi-Bridge/captions.txt"

# Open the output file to write the image paths and captions
with open(output_file, 'w', encoding='utf-8') as f_out:
    # Loop through each numbered subfolder
    for subdir in os.listdir(base_path):
        subfolder_path = os.path.join(base_path, subdir)

        # Check if the path is a directory
        if os.path.isdir(subfolder_path):
            print(f"Processing subfolder: {subfolder_path}")

            # List files in the current subfolder for debugging
            files_in_subfolder = os.listdir(subfolder_path)
            print(f"Files in {subfolder_path}: {files_in_subfolder}")

            image_path = os.path.join(subfolder_path, 'img.jpg')
            caption_path = os.path.join(subfolder_path, 'captions.txt')

            print(f"Looking for image: {image_path}")
            print(f"Looking for caption: {caption_path}")

            # Check if both files exist
            if os.path.exists(image_path) and os.path.exists(caption_path):
                try:
                    with open(caption_path, 'r', encoding='utf-8') as f_caption:
                        caption = f_caption.read().replace('\n', ' ').strip()

                        # Tokenize caption and check token length
                        tokens = caption.split()
                        if 10 <= len(tokens) <= 512:
                            # Write image path and valid caption to output file
                            f_out.write(f"{image_path}\t{caption}\n")
                            print(f"Found image and caption. Writing to file: {image_path}\t{caption}\n")
                        else:
                            print(f"Caption in {caption_path} does not meet token length requirements (10-512 tokens). SKIP THIS FILE.")

                except UnicodeDecodeError as e:
                    print(f"Error reading caption file {caption_path}: {e}")
            else:
                if not os.path.exists(image_path):
                    print(f"Image file does not exist: {image_path}")
                if not os.path.exists(caption_path):
                    print(f"Caption file does not exist: {caption_path}")

print(f"Captions file created at {output_file}")

# Step 2: Load the captions.txt into a DataFrame
input_file = output_file
df = pd.read_csv(input_file, delimiter='\t', header=None, names=['image_path', 'caption'])

# Check the DataFrame structure and contents
print("DataFrame structure:")
print(df.info())  # This will show the column names and types
print("DataFrame contents:")
print(df.head())

# Sample rows from the DataFrame if not empty
if not df.empty:
    sampled_df = df.sample(n=1)  # Sample a single random row
    print("Sampled DataFrame row:")
    print(sampled_df)
else:
    print("The DataFrame is empty. Check if captions.txt has valid entries.")


Processing subfolder: /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints
Files in /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints: []
Looking for image: /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints/img.jpg
Looking for caption: /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints/captions.txt
Image file does not exist: /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints/img.jpg
Caption file does not exist: /content/drive/My Drive/LaVi-Bridge/.ipynb_checkpoints/captions.txt
Captions file created at /content/drive/My Drive/LaVi-Bridge/captions.txt
DataFrame structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  0 non-null      object
 1   caption     0 non-null      object
dtypes: object(2)
memory usage: 124.0+ bytes
None
DataFrame contents:
Empty DataFrame
Columns: [image_path, caption]
Index: []
The DataFrame is empty.

In [13]:
# Check if the captions.txt file exists and read its contents
output_file = "/content/drive/My Drive/LaVi-Bridge/captions.txt"

# Check if the file exists
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        contents = f.read()
        print(contents)  # Print the contents to see if it's populated
else:
    print("File does not exist.")

In [14]:
print("Shape of DataFrame:", df.shape)
print(df.head())  # Show the first few rows

Shape of DataFrame: (0, 2)
Empty DataFrame
Columns: [image_path, caption]
Index: []


In [15]:
df.sample(0)

,image_path,caption


In [16]:

print("DataFrame Columns:", df.columns)
print("First few rows of DataFrame:")
print(df.head())

DataFrame Columns: Index(['image_path', 'caption'], dtype='object')
First few rows of DataFrame:
Empty DataFrame
Columns: [image_path, caption]
Index: []


In [17]:
for index, row in df.iterrows():
    print(f"Image Path: {row['image_path']}, Caption: {row['caption']}")

In [19]:
df["caption"] = "summarize: "+df["caption"]

In [18]:
df.head()

,image_path,caption


In [20]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [21]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [22]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [23]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [24]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150,
              num_beams=2,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [25]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rich.table import Column, Table
from rich import box
from rich.console import Console

# Global variables for tokenizer and model
tokenizer = None
model = None

# Define a rich console logger
console = Console(record=True)

def display_df(df):
    """Display dataframe in ASCII format."""
    console = Console()
    table = Table(Column("source_text", justify="center"), Column("target_text", justify="center"),
                  title="Sample Data", pad_edge=False, box=box.ASCII)

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/"):
    global tokenizer, model  # Declare tokenizer and model as global

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenizer for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"],
                                     model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
    val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"],
                                model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': model_params["TRAIN_BATCH_SIZE"],
        'shuffle': True,
        'num_workers': 0
    }

    val_params = {
        'batch_size': model_params["VALID_BATCH_SIZE"],
        'shuffle': False,
        'num_workers': 0
    }

    # Creation of Dataloaders
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer
    optimizer = torch.optim.Adam(params=model.parameters(), lr=model_params["LEARNING_RATE"])

    # Training loop
    console.log(f'[Initiating Fine Tuning]...\n')

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text': predictions, 'Actual Text': actuals})
        final_df.to_csv(os.path.join(output_dir, 'predictions.csv'))

    console.save_text(os.path.join(output_dir, 'logs.txt'))

    console.log(f"[Validation Completed.]\n")
    console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
    console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir, 'predictions.csv')}\n""")
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir, 'logs.txt')}\n""")


In [26]:
model_params={
    "MODEL":"t5-large",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":1,          # training batch size
    "VALID_BATCH_SIZE":1,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility

}

In [27]:
print(df.columns)

Index(['image_path', 'caption'], dtype='object')


In [28]:
df.rename(columns={'image_path': 'text', 'caption': 'headlines'}, inplace=True)

In [29]:
# Check the new column names
print("Renamed DataFrame columns:")
print(df.columns)

Renamed DataFrame columns:
Index(['text', 'headlines'], dtype='object')


In [30]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[01:03:35] [Model]: Loading t5-large...                                          <ipython-input-25-7a3b776ffcf2>:38
                                                                                                                   

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

        Sample Data        
+-------------------------+
|source_text | target_text|
|------------+------------|
+-------------------------+

FULL Dataset: (0, 2)

TRAIN Dataset: (0, 2)

TEST Dataset: (0, 2)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
print("Column names in df_predictions:", df.columns.tolist())

Column names in df_predictions: ['text', 'headlines']


In [ ]:
print(df.columns)

Index(['text', 'headlines'], dtype='object')


In [ ]:
if 'headlines' in df.columns:
    df['input'] = "Summarize this: " + df['headlines']
else:
    print("Caption column not found. Check the DataFrame structure.")

In [ ]:
df['input'] = "Summarize this: " + df['headlines']

In [ ]:
print(df['input'].apply(len))  # Check lengths of input strings


0    140
1    278
2    234
Name: input, dtype: int64


In [ ]:
print(df)

                                            text  \
0  /content/drive/My Drive/LaVi-Bridge/1/img.jpg   
1  /content/drive/My Drive/LaVi-Bridge/3/img.jpg   
2  /content/drive/My Drive/LaVi-Bridge/5/img.jpg   

                                           headlines  \
0  summarize: Start your day with the Sweet Heat ...   
1  summarize: Try it for yourself to find your id...   
2  summarize: Be part of a different kind of conv...   

                                               input  
0  Summarize this: summarize: Start your day with...  
1  Summarize this: summarize: Try it for yourself...  
2  Summarize this: summarize: Be part of a differ...  


In [ ]:
print(df['headlines'])
print(df['input'])

0    summarize: Start your day with the Sweet Heat ...
1    summarize: Try it for yourself to find your id...
2    summarize: Be part of a different kind of conv...
Name: headlines, dtype: object
0    Summarize this: summarize: Start your day with...
1    Summarize this: summarize: Try it for yourself...
2    Summarize this: summarize: Be part of a differ...
Name: input, dtype: object


In [ ]:
import pandas as pd

# Load the captions.txt into a DataFrame
input_file = "/content/drive/My Drive/LaVi-Bridge/captions.txt"
df_predictions = pd.read_csv(input_file, delimiter='\t', header=None, names=['image_path', 'caption'])

# Confirm the DataFrame was loaded correctly
print("DataFrame loaded. Contents:")
print(df_predictions.head())

DataFrame loaded. Contents:
                                      image_path  \
0  /content/drive/My Drive/LaVi-Bridge/1/img.jpg   
1  /content/drive/My Drive/LaVi-Bridge/3/img.jpg   
2  /content/drive/My Drive/LaVi-Bridge/5/img.jpg   

                                             caption  
0  Start your day with the Sweet Heat Breakfast B...  
1  Try it for yourself to find your ideal state o...  
2  Be part of a different kind of conversation — ...  


In [ ]:
import re
import torch

# Assuming model and tokenizer are defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to device

df_predictions['input'] = df_predictions['caption']

def generate_summary(text):
    try:
        inputs = tokenizer.encode(text, return_tensors='pt').to(device)
        summary_ids = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
        generated_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Remove consecutive duplicate phrases in generated text
        unique_summary = re.sub(r'\b(\w+\s+){2,}(\w+\s+)+\1\b', r'\1', generated_text.strip())

        # Truncate at the first occurrence of "!" or "..." -> this fixed the problem where the model would repeat itself over and over again about some generated text
        truncated_summary = re.split(r'(!|\.\.\.)', unique_summary, maxsplit=1)[0]

        # Split into sentences and get at least 2-3 sentences if possible
        sentences = re.split(r'(?<=[.!?]) +', truncated_summary)
        if len(sentences) < 3:
            # If fewer than 3 sentences, use them all; otherwise, limit to first 3
            final_summary = ' '.join(sentences)
        else:
            final_summary = ' '.join(sentences[:3])

        return final_summary.strip()

    except Exception as e:
        print(f"Error generating summary for text: {text}")
        print(f"Error: {e}")
        return ""  # Return an empty string in case of error

# Apply the summary generation
df_predictions['Generated Text'] = df_predictions['input'].apply(generate_summary)

# Now print the generated summaries
print("Generated Summaries and Actual Texts:")
for index, row in df_predictions.iterrows():
    generated_text = row['Generated Text'] if 'Generated Text' in df_predictions.columns else "Not Generated"
    actual_text = row['caption']  # or row['Actual Text'] if that was in your previous DataFrame
    print(f"Entry {index + 1}:")
    print(f"Generated Summary: {generated_text}")
    print(f"Actual Summary: {actual_text}\n")


Generated Summaries and Actual Texts:
Entry 1:
Generated Summary: sweet potatoes, avocado and more with Spicy Adobo Sauce
Actual Summary: Start your day with the Sweet Heat Breakfast Bowl: sweet potatoes, avocado and more with a new Spicy Adobo Sauce!

Entry 2:
Generated Summary: CLASSIC. Feel free CLASSIC is a feel-good tonic for your feel free state of mind.. Not for children.
Actual Summary: Try it for yourself to find your ideal state of "chilled energy."  Made with botanicals sourced from the South Pacific and Southeast Asia, 𝒇𝒆𝒆𝒍 𝒇𝒓𝒆𝒆 CLASSIC is a feel-good tonic for your 𝒇𝒆𝒆𝒍 𝒇𝒓𝒆𝒆 state of mind.   Consume responsibly. Adults 21+ Only.

Entry 3:
Generated Summary: .. Be part of a different conversation — that unite us
Actual Summary: Be part of a different kind of conversation — one that welcomes every perspective in a search for the deeper truths that unite us. Attend free spiritual discussions and classes with others in your community.

